### Library

In [96]:
%pip install requests
%pip install BeautifulSoup4

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [97]:
import requests
from bs4 import BeautifulSoup

import pandas as pd
import numpy as np

# import textblob
from textblob import Word
from textblob import TextBlob

In [98]:
r = requests.get("https://www.yelp.com/biz/kia-of-marin-novato")
soup = BeautifulSoup(r.text, "html.parser")

In [99]:
r.status_code

200

## Pulling data from the internet using BeautifulSoup

In [100]:
def create_data():
    comments =  []
    userName = []
    rating = []
    
    
    soup_lis = soup \
    .find('ul', {'class': 'list__09f24__ynIEd'})\
    .find_next('li', {'class': 'y-css-1jp2syp'})\
    
    
    #Yorumlar kısmını çekme
    for lis in soup_lis:
        li = lis.find_next("div", {'class': 'y-css-1iy1dwt'})\
            .find_next("div", {'class': 'y-css-cluvhg'})\
            .find_all_next("p", {'class': 'comment__09f24__D0cxf y-css-h9c2fl'})
            
        for l in li:
            comments.append(l.text)
    
    #Kullanici adlarını çekme    
    for lis in soup_lis:
        li = lis.find_next("div", {'class': 'y-css-1iy1dwt'})\
            .find_next("span", {'class': 'y-css-w3ea6v'})\
            .find_all_next("a", {"class": "y-css-12ly5yx"})
            
        for l in li:
            if l.text != "1 photo":
                userName.append(l.text)

    #Kaç yıldız verdiklerini görürüz. Onları çekeriz.
    for lis in soup_lis:
        li = lis.find_next("div", {'class': 'y-css-1iy1dwt'})\
            .find_next("div", {'class': 'y-css-19pbem2'})\
            .find_next("span", {'class': 'y-css-pw0opj'})\
            .find_all_next("div", attrs={"class": 'y-css-9tnml4'})
            
            
        for l in li:
            #print(l['aria-label'])
            rating.append(l['aria-label'])
        
        rating = rating[:10]

    return comments, userName, rating

In [101]:
comments, userName, rating = create_data()

## Analysis The Data

In [102]:
data = pd.DataFrame(userName, columns=["UserName"])
data["Review"] = pd.DataFrame(comments, columns=["Review"])
data["Rating"] = pd.DataFrame(rating, columns=["Rating"])

In [103]:
data.head()

,UserName,Review,Rating
0,Maureen P.,Even an EV needs service from time to time. Ki...,5 star rating
1,Isis L.,I wasn't looking to buy a car as I was content...,5 star rating
2,Patrick H.,Worked with Julio and he was excellent from st...,5 star rating
3,Koichi D.,Been dealing with Kia Marin for years. Leased ...,5 star rating
4,Janessa B.,Been bringing my Kia k5 here for a couple of y...,5 star rating


In [104]:
len(data['Review'])

10

In [105]:
data_word = pd.DataFrame(data["Review"]) 

In [106]:
data_word

,Review
0,Even an EV needs service from time to time. Ki...
1,I wasn't looking to buy a car as I was content...
2,Worked with Julio and he was excellent from st...
3,Been dealing with Kia Marin for years. Leased ...
4,Been bringing my Kia k5 here for a couple of y...
5,Unprofessional sales team. They dismiss conver...
6,I bought a new Kia Sorento Hybrid from Kia of ...
7,I recently purchased a used car from this deal...
8,Best car buying experience. Julio was awesom...
9,I usually only post reviews on Yelp when I hav...


In [107]:
data_word["Word_count"] = data_word['Review'].apply(lambda x: len(x.split()))

In [108]:
data_word["Char_count"] = data_word['Review'].apply(lambda x: len(x))

In [109]:
import nltk
from nltk.corpus import stopwords

In [110]:
stop_words = stopwords.words("english")

In [111]:
len(stop_words)

179

In [112]:
data_word["stopword_counter"] = data_word["Review"].apply(lambda x: len([word for word in x.split() if word.lower() in stop_words]))

In [113]:
data_word["stopword_rate"] = data_word["stopword_counter"] / data_word["Word_count"]

In [114]:
data.head()

,UserName,Review,Rating
0,Maureen P.,Even an EV needs service from time to time. Ki...,5 star rating
1,Isis L.,I wasn't looking to buy a car as I was content...,5 star rating
2,Patrick H.,Worked with Julio and he was excellent from st...,5 star rating
3,Koichi D.,Been dealing with Kia Marin for years. Leased ...,5 star rating
4,Janessa B.,Been bringing my Kia k5 here for a couple of y...,5 star rating


In [115]:
data_word

,Review,Word_count,Char_count,stopword_counter,stopword_rate
0,Even an EV needs service from time to time. Ki...,172,948,76,0.441860
1,I wasn't looking to buy a car as I was content...,347,1691,164,0.472622
2,Worked with Julio and he was excellent from st...,89,511,44,0.494382
3,Been dealing with Kia Marin for years. Leased ...,84,473,36,0.428571
4,Been bringing my Kia k5 here for a couple of y...,91,473,47,0.516484
5,Unprofessional sales team. They dismiss conver...,53,355,22,0.415094
6,I bought a new Kia Sorento Hybrid from Kia of ...,110,564,57,0.518182
7,I recently purchased a used car from this deal...,136,758,65,0.477941
8,Best car buying experience. Julio was awesom...,39,258,13,0.333333
9,I usually only post reviews on Yelp when I hav...,186,998,88,0.473118


In [116]:
data_word.sort_values(by = "stopword_rate")

,Review,Word_count,Char_count,stopword_counter,stopword_rate
8,Best car buying experience. Julio was awesom...,39,258,13,0.333333
5,Unprofessional sales team. They dismiss conver...,53,355,22,0.415094
3,Been dealing with Kia Marin for years. Leased ...,84,473,36,0.428571
0,Even an EV needs service from time to time. Ki...,172,948,76,0.441860
1,I wasn't looking to buy a car as I was content...,347,1691,164,0.472622
9,I usually only post reviews on Yelp when I hav...,186,998,88,0.473118
7,I recently purchased a used car from this deal...,136,758,65,0.477941
2,Worked with Julio and he was excellent from st...,89,511,44,0.494382
4,Been bringing my Kia k5 here for a couple of y...,91,473,47,0.516484
6,I bought a new Kia Sorento Hybrid from Kia of ...,110,564,57,0.518182


In [117]:
data_word.describe()

,Word_count,Char_count,stopword_counter,stopword_rate
count,10.000000,10.000000,10.000000,10.000000
mean,130.700000,702.900000,61.200000,0.457159
std,89.410477,422.602111,42.855831,0.055486
min,39.000000,258.000000,13.000000,0.333333
25%,85.250000,473.000000,38.000000,0.431894
50%,100.500000,537.500000,52.000000,0.472870
75%,163.000000,900.500000,73.250000,0.490272
max,347.000000,1691.000000,164.000000,0.518182


## Data Cleaning

In [118]:
data_word["lowercase"] = data_word["Review"].apply(lambda x: " ".join(word.lower() for word in x.split()))
data_word["punction"] = data_word["lowercase"].str.replace("[^\w\s]", "")
data_word["stopword"] = data_word["punction"].apply(lambda x: " ".join(word for word in x.split() if word not in stop_words))

In [119]:
data_word.head()

,Review,Word_count,Char_count,stopword_counter,stopword_rate,lowercase,punction,stopword
0,Even an EV needs service from time to time. Ki...,172,948,76,0.441860,even an ev needs service from time to time. ki...,even an ev needs service from time to time. ki...,even ev needs service time time. kia marin hit...
1,I wasn't looking to buy a car as I was content...,347,1691,164,0.472622,i wasn't looking to buy a car as i was content...,i wasn't looking to buy a car as i was content...,looking buy car content ny 2007 toyota yaris a...
2,Worked with Julio and he was excellent from st...,89,511,44,0.494382,worked with julio and he was excellent from st...,worked with julio and he was excellent from st...,worked julio excellent start finish. knowledge...
3,Been dealing with Kia Marin for years. Leased ...,84,473,36,0.428571,been dealing with kia marin for years. leased ...,been dealing with kia marin for years. leased ...,dealing kia marin years. leased many kia's yea...
4,Been bringing my Kia k5 here for a couple of y...,91,473,47,0.516484,been bringing my kia k5 here for a couple of y...,been bringing my kia k5 here for a couple of y...,"bringing kia k5 couple years must say, service..."


In [120]:
pd.Series("".join(data_word["stopword"]).split()).value_counts()[:20]

kia           15
car           14
service        8
experience     6
time           6
key            6
hugh           5
one            5
used           4
call           4
made           4
many           4
going          4
new            4
ev             4
within         3
buying         3
car.           3
got            3
help           3
Name: count, dtype: int64

## Lemmatization

In [121]:
data_word["lemmatize"] = data_word["stopword"].apply(lambda x: " ".join(Word(word).lemmatize() for word in x.split()))

## Sentiment Analysis

In [122]:
data_word["polarity"] = data_word["lemmatize"].apply(lambda x: TextBlob(x).sentiment[0])
data_word["subjectivity"] = data_word["lemmatize"].apply(lambda x: TextBlob(x).sentiment[1])

In [123]:
data_word.head()

,Review,Word_count,Char_count,stopword_counter,stopword_rate,lowercase,punction,stopword,lemmatize,polarity,subjectivity
0,Even an EV needs service from time to time. Ki...,172,948,76,0.441860,even an ev needs service from time to time. ki...,even an ev needs service from time to time. ki...,even ev needs service time time. kia marin hit...,even ev need service time time. kia marin hit ...,0.394444,0.669306
1,I wasn't looking to buy a car as I was content...,347,1691,164,0.472622,i wasn't looking to buy a car as i was content...,i wasn't looking to buy a car as i was content...,looking buy car content ny 2007 toyota yaris a...,looking buy car content ny 2007 toyota yaris a...,0.318008,0.611127
2,Worked with Julio and he was excellent from st...,89,511,44,0.494382,worked with julio and he was excellent from st...,worked with julio and he was excellent from st...,worked julio excellent start finish. knowledge...,worked julio excellent start finish. knowledge...,0.475000,0.665625
3,Been dealing with Kia Marin for years. Leased ...,84,473,36,0.428571,been dealing with kia marin for years. leased ...,been dealing with kia marin for years. leased ...,dealing kia marin years. leased many kia's yea...,dealing kia marin years. leased many kia's yea...,0.307589,0.606200
4,Been bringing my Kia k5 here for a couple of y...,91,473,47,0.516484,been bringing my kia k5 here for a couple of y...,been bringing my kia k5 here for a couple of y...,"bringing kia k5 couple years must say, service...","bringing kia k5 couple year must say, service ...",0.313542,0.515625


In [124]:
data_word.columns

Index(['Review', 'Word_count', 'Char_count', 'stopword_counter',
       'stopword_rate', 'lowercase', 'punction', 'stopword', 'lemmatize',
       'polarity', 'subjectivity'],
      dtype='object')

In [125]:
data_word.drop(['lowercase', 'punction', 'stopword', 'lemmatize'], axis=1, inplace=True)

In [126]:
data_word.head()

,Review,Word_count,Char_count,stopword_counter,stopword_rate,polarity,subjectivity
0,Even an EV needs service from time to time. Ki...,172,948,76,0.441860,0.394444,0.669306
1,I wasn't looking to buy a car as I was content...,347,1691,164,0.472622,0.318008,0.611127
2,Worked with Julio and he was excellent from st...,89,511,44,0.494382,0.475000,0.665625
3,Been dealing with Kia Marin for years. Leased ...,84,473,36,0.428571,0.307589,0.606200
4,Been bringing my Kia k5 here for a couple of y...,91,473,47,0.516484,0.313542,0.515625


In [127]:
data_word.sort_values(by="polarity")

,Review,Word_count,Char_count,stopword_counter,stopword_rate,polarity,subjectivity
9,I usually only post reviews on Yelp when I hav...,186,998,88,0.473118,0.008117,0.539881
5,Unprofessional sales team. They dismiss conver...,53,355,22,0.415094,0.016667,0.483333
7,I recently purchased a used car from this deal...,136,758,65,0.477941,0.135714,0.602778
6,I bought a new Kia Sorento Hybrid from Kia of ...,110,564,57,0.518182,0.275118,0.535458
3,Been dealing with Kia Marin for years. Leased ...,84,473,36,0.428571,0.307589,0.606200
4,Been bringing my Kia k5 here for a couple of y...,91,473,47,0.516484,0.313542,0.515625
1,I wasn't looking to buy a car as I was content...,347,1691,164,0.472622,0.318008,0.611127
0,Even an EV needs service from time to time. Ki...,172,948,76,0.441860,0.394444,0.669306
2,Worked with Julio and he was excellent from st...,89,511,44,0.494382,0.475000,0.665625
8,Best car buying experience. Julio was awesom...,39,258,13,0.333333,0.539167,0.669167


# Sentiment Analysis with BERT Neural Network and Python

## Plan
1. Install Transformers
2. Perform Sentiment Analysis Scoring using BERT
3. Scrape review from YELP and score

### 1. Install and Import Dependencies

In [128]:
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

Looking in indexes: https://download.pytorch.org/whl/cu118


In [129]:
!pip install transformers requests beautifulsoup4 pandas numpy

In [130]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

### 2. Instantiate Model

In [131]:
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')
model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

### 3. Collect Review

In [132]:
tokens = tokenizer.encode('Meh, it was okey!', return_tensors="pt")
result = model(tokens)


In [133]:
result.logits

tensor([[-0.7738,  1.8016,  2.8773, -0.3960, -2.9344]],
       grad_fn=<AddmmBackward0>)

In [134]:
int(torch.argmax(result.logits))+1

3

### 4. Load Reviews into DataFrame and Score

In [135]:
data["Rating"] = data["Rating"].apply(lambda x: ''.join(num for num in x.split() if num.isnumeric()))

In [136]:
data

,UserName,Review,Rating
0,Maureen P.,Even an EV needs service from time to time. Ki...,5
1,Isis L.,I wasn't looking to buy a car as I was content...,5
2,Patrick H.,Worked with Julio and he was excellent from st...,5
3,Koichi D.,Been dealing with Kia Marin for years. Leased ...,5
4,Janessa B.,Been bringing my Kia k5 here for a couple of y...,5
5,Vikrant B.,Unprofessional sales team. They dismiss conver...,1
6,Amanda Y.,I bought a new Kia Sorento Hybrid from Kia of ...,5
7,Francis K.,I recently purchased a used car from this deal...,4
8,Jay C.,Best car buying experience. Julio was awesom...,5
9,Michael F.,I usually only post reviews on Yelp when I hav...,1


In [137]:
data["Review"].iloc[0]

"Even an EV needs service from time to time. Kia Marin hits the ball out of the park each time I've needed assistance or service. I purchased my Kia Niro EV in 2019 and couldn't be happier. Matthew, Eddie and the entire team are efficient, knowledgeable, fully up to speed on Kia vehicle services and explain services and pricing so you know what you are getting is what your car needs at a reasonable price. I  know I will get excellent service and a pleasant welcome each time I call or come by in person. I am very happy with my vehicle and its care. This uncomplicates at least one important job in daily life - having safe, reliable and well maintained transportation. Would that Kia could branch out to other endeavors to help our lives flow smoothly. Go with Kia. No wonder it gets high  marks in national vehicle reviews! Not just the product is the end of the line. It's a well build service organization that cares about keeping customers."

In [138]:
def sentiment_score(review):
    tokens = tokenizer.encode(review, return_tensors="pt")
    result = model(tokens)
    return int(torch.argmax(result.logits)) + 1

In [139]:
sentiment_score(data["Review"].iloc[0])

5

In [140]:
data["Sentiment"] = data["Review"].apply(lambda x: sentiment_score(x))

In [141]:
data

,UserName,Review,Rating,Sentiment
0,Maureen P.,Even an EV needs service from time to time. Ki...,5,5
1,Isis L.,I wasn't looking to buy a car as I was content...,5,4
2,Patrick H.,Worked with Julio and he was excellent from st...,5,5
3,Koichi D.,Been dealing with Kia Marin for years. Leased ...,5,5
4,Janessa B.,Been bringing my Kia k5 here for a couple of y...,5,5
5,Vikrant B.,Unprofessional sales team. They dismiss conver...,1,1
6,Amanda Y.,I bought a new Kia Sorento Hybrid from Kia of ...,5,5
7,Francis K.,I recently purchased a used car from this deal...,4,1
8,Jay C.,Best car buying experience. Julio was awesom...,5,5
9,Michael F.,I usually only post reviews on Yelp when I hav...,1,2
